In [1]:
import os
import numpy as np
import pandas as pd
import scipy.io
from utils import count_labels
# 忽略警告
np.warnings.filterwarnings('ignore')

# 定义采样频率
fs = 300

# 数据目录路径
data_dir = 'C:\\Users\\Administrator\\Desktop\\database\\training2017\\'

# 读取csv注释文件获取标签
ref = pd.read_csv(data_dir + 'REFERENCE-original.csv', names=['record', 'rhythm'])
rhythm2 = np.array(ref['rhythm'])  # 分类变成Non-AF和AF

# 创建字典表格
ref_dict = dict(zip(ref['record'], ref['rhythm']))
ref2_dict = dict(zip(ref['record'], rhythm2))

# 从文件路径中提取文件名，不包含后缀
base = lambda f: os.path.splitext(os.path.basename(f))[0]

# 提取文件名
files = ref['record']

# 初始化列表
ecgs = []
SID = []
ecgs1 = []
labels = []
labels2 = []



c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
for f in files:
    ecgdata = scipy.io.loadmat(data_dir + f)['val']
    ecgdata = ecgdata.ravel()  
    ecgs.append(ecgdata)
    labels.append(ref_dict[base(f)]) 
    labels2.append(ref2_dict[base(f)])  
    SID.append(base(f))
print(pd.DataFrame(zip(SID, labels2, labels), columns=['SubjectID', '2 Class Label', '3 Class Label']))


     SubjectID 2 Class Label 3 Class Label
0       A00001             N             N
1       A00002             N             N
2       A00003             N             N
3       A00004             A             A
4       A00005             O             O
...        ...           ...           ...
8523    A08524             N             N
8524    A08525             O             O
8525    A08526             N             N
8526    A08527             N             N
8527    A08528             N             N

[8528 rows x 3 columns]


In [3]:
import numpy as np
import random

def segment_signals(ecgs, labels):
    # 初始化存储最终处理后的信号和标签的列表
    ecgs_final = []
    labels_final = []

    # 随机打乱数据集
    dataset = list(zip(ecgs, labels))
    random.shuffle(dataset)
    ecgs, labels = zip(*dataset)

    # 处理数据集，将大于14000的信号切割成两半
    for ecgdata, label in zip(ecgs, labels):
        if len(ecgdata) > 14000:
            # 切割成两半，每半小于等于14000
            half_len = len(ecgdata) // 2
            first_half = ecgdata[:half_len]
            second_half = ecgdata[half_len:]
            ecgs_final.extend([first_half, second_half])
            labels_final.extend([label] * 2)
        else:
            ecgs_final.append(ecgdata)
            labels_final.append(label)

    # 处理数据集，将大于7000的信号切割成两半
    # 这里需要重新收集所有ecgs_final的数据，因为列表在前面被修改过
    ecgs, labels = zip(*[(ecg, label) for ecg, label in zip(ecgs_final, labels_final) if len(ecg) > 7000])

    # 将大于7000的信号切割成两半
    for i, (ecgdata, label) in enumerate(zip(ecgs_final, labels_final)):
        if len(ecgdata) > 7000:
            half_len = 7000  # 设置切割长度为7000
            first_half = ecgdata[:half_len]
            second_half = ecgdata[-half_len:]  # 取后半段7000
            # 替换原来的信号和标签
            ecgs_final[i] = first_half
            ecgs_final.insert(i + 1, second_half)
            labels_final[i] = label
            labels_final.insert(i + 1, label)

    # 调整所有信号到4500长度
    for i, ecgdata in enumerate(ecgs_final):
        if len(ecgdata) > 4500:
            # 如果信号长度大于4500，随机截取4500个样本点
            start_index = np.random.randint(0, len(ecgdata) - 4500 + 1)
            ecgs_final[i] = ecgdata[start_index:start_index + 4500]
        elif len(ecgdata) < 4500:
            # 如果信号长度小于4500，使用信号自身的片段补齐至4500
            repeat_times = (4500 // len(ecgdata)) + (1 if 4500 % len(ecgdata) else 0)
            additional_ecg = np.tile(ecgdata, (repeat_times,))
            ecgs_final[i] = additional_ecg[:4500]

    return ecgs_final, labels_final

# 使用示例
# 假设 ecgs 和 labels 是您的原始信号和标签列表
ecgs1, labels1 = segment_signals(ecgs, labels)
print(len(ecgs1), len(labels1))


def merge_and_segment(ecgs, labels, threshold):
    merged_ecgs = []
    merged_labels = []
    for ecgdata, label in zip(ecgs, labels):
        if len(ecgdata) > threshold:
            first_half = ecgdata[:threshold]
            second_half = ecgdata[-threshold:]
            merged_ecgs.extend([first_half, second_half])
            merged_labels.extend([label] * 2)
        else:
            merged_ecgs.append(ecgdata)
            merged_labels.append(label)
    return merged_ecgs, merged_labels
def adjust_to_4500(ecgs, labels):
    ecgs_4500 = []
    for ecgdata, label in zip(ecgs, labels):
        if len(ecgdata) > 4500:
            start_index = np.random.randint(0, len(ecgdata) - 4500)
            trimmed_ecg = ecgdata[start_index:start_index + 4500]
            ecgs_4500.append(trimmed_ecg)
        elif len(ecgdata) == 4500:
            ecgs_4500.append(ecgdata)
        else:
            additional_ecg = select_additional_ecg(label, ecgs, labels)
            repeat_times = 4500 // len(ecgdata)
            additional_ecg = np.tile(ecgdata, (repeat_times,)) if repeat_times > 0 else np.array([])
            remainder = 4500 - len(additional_ecg)
            ecgs_4500.append(np.concatenate((additional_ecg, ecgdata[:remainder])))
    return ecgs_4500
def select_additional_ecg(label, ecgs, labels):
    same_label_ecgs = [e for e, l in zip(ecgs, labels) if l == label]
    if not same_label_ecgs:
        return np.array([]) 
    longest_ecg = max(same_label_ecgs, key=len)
    return longest_ecg
def select_additional_ecg(ecgdata, max_length):
    repeat_times = (max_length // len(ecgdata)) + (1 if max_length % len(ecgdata) else 0)
    additional_ecg = np.tile(ecgdata, (repeat_times,))
    return additional_ecg[:max_length]

18347 18347


In [4]:
ecgs1, labels1 = segment_signals(ecgs, labels)
print(len(ecgs1), len(labels1))

18347 18347


In [5]:
count_labels(labels1)

{'A': 1630, 'N': 10865, 'O': 5771, '~': 81}

In [6]:
np.savez('cinc2017_seg8.npz', ecgs=ecgs1, labels=labels1)